<a href="https://colab.research.google.com/github/erikgqp8645/-Pandas-/blob/master/%E5%9B%BA%E5%AE%9A%E5%B8%82%E5%80%BC%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
!pip install akshare

In [81]:
import pandas as pd
import akshare as ak

In [82]:
df = ak.fund_open_fund_info_em(fund="710001", indicator="单位净值走势")
df

,净值日期,单位净值,日增长率
0,2011-09-21,1.0000,0.00
1,2011-09-23,1.0000,0.00
2,2011-09-30,1.0001,0.01
3,2011-10-14,1.0005,0.04
4,2011-10-21,1.0007,0.02
...,...,...,...
2524,2022-02-23,3.4131,0.70
2525,2022-02-24,3.3576,-1.63
2526,2022-02-25,3.3492,-0.25
2527,2022-02-28,3.3406,-0.26


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2529 entries, 0 to 2528
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   净值日期    2529 non-null   object 
 1   单位净值    2529 non-null   float64
 2   日增长率    2529 non-null   float64
dtypes: float64(2), object(1)
memory usage: 59.4+ KB


In [84]:
res = (
    df.astype({"净值日期":"datetime64[ns]"})
    .set_index("净值日期")
    .assign(月份 = lambda m :m.index.month)
    .assign(日 = lambda m :m.index.day)
    
)

In [85]:
res

,单位净值,日增长率,月份,日
净值日期,,,,
2011-09-21,1.0000,0.00,9,21
2011-09-23,1.0000,0.00,9,23
2011-09-30,1.0001,0.01,9,30
2011-10-14,1.0005,0.04,10,14
2011-10-21,1.0007,0.02,10,21
...,...,...,...,...
2022-02-23,3.4131,0.70,2,23
2022-02-24,3.3576,-1.63,2,24
2022-02-25,3.3492,-0.25,2,25


In [86]:
res["最大日期"] =res.groupby("月份")["日"].transform("max")

In [87]:
res

,单位净值,日增长率,月份,日,最大日期
净值日期,,,,,
2011-09-21,1.0000,0.00,9,21,30
2011-09-23,1.0000,0.00,9,23,30
2011-09-30,1.0001,0.01,9,30,30
2011-10-14,1.0005,0.04,10,14,31
2011-10-21,1.0007,0.02,10,21,31
...,...,...,...,...,...
2022-02-23,3.4131,0.70,2,23,29
2022-02-24,3.3576,-1.63,2,24,29
2022-02-25,3.3492,-0.25,2,25,29


In [88]:
res = res[res['日']==res['最大日期']]
res

,单位净值,日增长率,月份,日,最大日期
净值日期,,,,,
2011-09-30,1.0001,0.0100,9,30,30
2011-10-31,1.0107,0.5572,10,31,31
2011-11-30,0.9849,-3.3654,11,30,30
2011-12-31,0.9539,0.0000,12,31,31
2012-01-31,0.9185,0.0436,1,31,31
...,...,...,...,...,...
2021-06-30,3.4880,1.6200,6,30,30
2021-08-31,3.6250,-0.5100,8,31,31
2021-09-30,3.4974,2.2200,9,30,30


In [96]:
(
    res.assign(计划投资 = 1000)
    .assign(日增长率=res.单位净值.pct_change())
    .assign(累计投资 = lambda x :x.计划投资.cumsum())
    .assign(上月张跌后市值 = lambda y:y.累计投资.shift(1)*(1+y.日增长率))
    .assign(实际投入金额 = lambda s:s.累计投资-s.上月张跌后市值)
    .fillna({"上月张跌后市值":1000,"实际投入金额":1000})
)

,单位净值,日增长率,月份,日,最大日期,计划投资,累计投资,上月张跌后市值,实际投入金额
净值日期,,,,,,,,,
2011-09-30,1.0001,NaN,9,30,30,1000,1000,1000.000000,1000.000000
2011-10-31,1.0107,0.010599,10,31,31,1000,2000,1010.598940,989.401060
2011-11-30,0.9849,-0.025527,11,30,30,1000,3000,1948.946275,1051.053725
2011-12-31,0.9539,-0.031475,12,31,31,1000,4000,2905.574170,1094.425830
2012-01-31,0.9185,-0.037111,1,31,31,1000,5000,3851.556767,1148.443233
...,...,...,...,...,...,...,...,...,...
2021-06-30,3.4880,0.084779,6,30,30,1000,82000,87867.139392,-5867.139392
2021-08-31,3.6250,0.039278,8,31,31,1000,83000,85220.756881,-2220.756881
2021-09-30,3.4974,-0.035200,9,30,30,1000,84000,80078.400000,3921.600000
